# TODO 1) Owner'ların işletme randevusu ekleyebilmesi (Staff bazlı ve servis süreli) — GMT+3 sabit

Amaç
- Owner (business_owner), kendi işletmeleri için müşteriler adına randevu oluşturabilsin.
- Müşteri personel seçmez; sistem, seçilen servisi yapabilen ve o anda uygun olan bir personeli OTOMATİK atar (staff_id zorunlu fakat sistem tarafından belirlenir).
- Farklı servislerin farklı SÜRELERİ dikkate alınır; slotlar saat başı değil, slot aralığına (örn. 15 dk) uyumlu hesaplanır.
- Tüm saat hesapları sabit olarak GMT+3 varsayımıyla yapılır (ayrı bir timezone ayarı YOK).

Kapsam
- Backend: Yetkilendirme, sahiplik (ownership) doğrulaması, uygunluk kontrolü, otomatik staff ataması, transaction ile çakışma önleme.
- Frontend (Owner paneli): “Yeni Randevu Ekle” formu; servis/datetime seçiminde uygun slotların gösterilmesi; kullanıcı personel seçmeyecek.
- Kayıt: Randevular normal müşteri randevuları gibi listelenir; kaynağı “owner_manual” olarak işaretlenir.

Önkoşullar (Şema ve Ayarlar)
- appointments tablosu şu alanları içermelidir:
  - business_id, service_id, customer_id (opsiyonel), customer_name, customer_phone, appointment_date (YYYY-MM-DD), start_time (HH:MM), end_time (HH:MM), staff_id, status, notes, source ('owner_manual' | 'customer'), created_at
- staff ve staff_services tabloları:
  - staff(business_id, name, active)
  - staff_services(staff_id, service_id)
- business_settings:
  - slot_interval_minutes (örn. 15)
  - min_notice_minutes (örn. 0–60)
  - booking_window_days (örn. 30)
- services.duration (dakika) servis süresi.

Gerekli Alanlar (Owner randevu formu)
- business_id (owner’a ait)
- service_id (işletmeye ait)
- appointment_date (YYYY-MM-DD)
- start_time (HH:MM) — slot aralığına uygun
- customer_name (metin)
- customer_phone (metin)
- notes (opsiyonel)

İş Kuralları ve Doğrulamalar
- Yetki:
  - Sadece role === 'business_owner' randevu oluşturabilir.
  - Sadece kendisine ait business_id’ler için randevu oluşturabilir (ownership check).
- Servis/Uygunluk:
  - service_id, business_id’ye ait olmalıdır.
  - need_minutes = services.duration (buffer YOK)
  - start_time + need_minutes, işletme kapanış saatini aşmamalıdır.
  - start_time, slot_interval_minutes gridine uymalıdır (örn. 09:00, 09:15, 09:30).
  - min_notice_minutes kuralı: Bugün için slot başlangıcı, “şu an (GMT+3) + min_notice” sonrasına denk gelmelidir. (Owner için opsiyonel override eklenebilir.)
- Personel Atama (otomatik):
  - staff_id formdan gelmez; backend, service_id’yi yapabilen ve seçilen zaman aralığında boş olan aktif bir staff’ı seçer.
  - Seçim stratejisi: “ilk uygun” veya “o gün en az randevusu olan”.
  - Hiç uygun staff yoksa 409 Conflict: “No available staff for this slot”.
- Çakışma Önleme:
  - Transaction (BEGIN IMMEDIATE) içinde, seçilen staff için slot çakışması tekrar kontrol edilir; uygunsa INSERT, aksi halde ROLLBACK + 409.
  - Çakışma kriteri: yarı-açık aralık [start, end) için overlap: (aStart < bEnd) && (bStart < aEnd).
- Kaynak Etiketi:
  - source = 'owner_manual' olarak kaydedilir; UI listelerinde bu bilgi gösterilebilir.
- Durumlar:
  - status: 'pending' veya doğrudan 'confirmed' (tercihe göre).

API Tasarımı

1) Uygunluk Gösterimi (Owner formu için)
- GET /api/businesses/:id/availability?service_id=...&date=YYYY-MM-DD
- Response örneği:
```json
{
  "date": "2025-11-06",
  "slots": [
    { "time": "13:15", "available_count": 2 },
    { "time": "13:30", "available_count": 1 }
  ]
}
```
Not: staff_id listesi döndürülmez; yalnızca “uygun personel sayısı” bilgisi gösterilir.

2) Randevu Oluşturma (Owner)
- POST /api/appointments
- Body:
```json
{
  "business_id": 12,
  "service_id": 5,
  "appointment_date": "2025-11-06",
  "start_time": "13:15",
  "customer_name": "Ada Kaya",
  "customer_phone": "+90 5xx xxx xx xx",
  "notes": "Telefonla alındı"
}
```
- Davranış:
  - Auth: business_owner
  - Ownership: business_id owner’a ait mi?
  - Uygunluk: service_id işletmeye ait mi? Slot grid, süre, kapanış, min_notice kontrolü
  - Otomatik atama: uygun staff seç
  - Transaction: re-check + INSERT
  - Response 201:
```json
{
  "appointment_id": 987,
  "staff_id": 8,
  "start_time": "13:15",
  "end_time": "14:00",
  "status": "pending",
  "source": "owner_manual"
}
```

Frontend (Owner Paneli) — Akış
1) İşletme seçimi: Owner’ın sadece kendi işletmeleri listelenir.
2) Servis seçimi: Seçilen işletmenin servisleri (süre ve ücret görünür).
3) Tarih seçimi: booking_window_days ile sınırla.
4) Saat seçimi:
   - Availability endpoint’inden dönen slot listesi gösterilir.
   - “13:15 — 2 uygun personel” gibi rozet; personel seçimi yok.
   - Seçimde, yaklaşık bitiş saati (yalnızca duration) gösterilir.
5) Müşteri bilgileri: customer_name, customer_phone (opsiyonel e-posta), notes.
6) Gönder: Başarılı ise listeye ekle; kaynağı “Owner” etiketiyle gör.

Kabul Kriterleri
- Backend
  - [ ] Owner sadece kendi business_id’leri için POST /api/appointments çağrısını yapabilir; diğer business_id için 403.
  - [ ] Slot üretimi servis süresi ile; slot_interval gridine uyumlu.
  - [ ] Otomatik staff ataması yapılır; staff_id randevu kaydında zorunlu.
  - [ ] Çakışma önleme transaction ile re-check; eşzamanlı isteklerde en fazla biri başarılı.
  - [ ] source='owner_manual' kayıt edilir.
  - [ ] status başlangıç değeri gereksinime göre 'pending' veya 'confirmed'.
- Frontend
  - [ ] Owner panelinde “Yeni Randevu Ekle” formu mevcut.
  - [ ] Saat seçiminde availability gösterilir; personel seçimi yoktur.
  - [ ] Kaydedilen randevu, doğru işletme ve atanan personel ile listede görünür; “Owner” etiketi gösterilir.
- Güvenlik
  - [ ] Auth zorunlu; role === 'business_owner'.
  - [ ] Ownership kontrolü zorunlu; cross-business denemeler engellenir.
  - [ ] İstek validasyonları ve hata kodları (400/403/409) doğru döner.

Kenar Durumları
- Uygun personel bulunamaz: 409.
- Servis süresi kapanışa taşarsa: 400.
- Min notice ihlali: 400 (owner override açık ise esnetilebilir).
- Personel pasif (active=0): otomatik atamada hesaba katılmaz.
- Eşzamanlı owner isteği: Transaction re-check ile tutarlılık.

Test Senaryoları (Örnek)
- [ ] Owner kendi işletmesi için başarılı randevu (201).
- [ ] Owner başka işletme için deneme (403).
- [ ] Uygun staff yok (409).
- [ ] Slot grid dışında start_time (400).
- [ ] Kapanışa sarkan randevu (400).
- [ ] Aynı slota iki eşzamanlı istek: biri 201, diğeri 409.
- [ ] source alanı 'owner_manual' olarak kaydediliyor.

Notlar
- Müşteri uygulamada yoksa customer_id NULL kalabilir; customer_name/phone zorunludur.
- Otomatik atama politikası ileride “en az meşgul” gibi geliştirilebilir.
- Tüm saat hesapları sabit GMT+3 kabulü ile yapılır; ayrı bir timezone alanı kullanılmaz.

# TODO 2) İşletme randevu saatleri ayarlanması — GMT+3 sabit

### Otomatik Personel Atamalı Randevu Sistemi (Staff_id Zorunlu)

Amaç
- Müşteri personel seçmeyecek; sistem, uygun ve ilgili hizmeti yapabilen personele otomatik atama yapacak.
- Aynı anda birden fazla randevu mümkün, ancak her hizmet için o hizmeti yapabilen personelin anlık kapasitesi kadar.
- Tüm saat hesapları sabit GMT+3 varsayımıyla yapılır (timezone alanı yok).

Özet Akış
1) İşletme ayarları ve personel-yetenekleri tanımlanır.
2) Müşteri: İşletme → Servis → Tarih → Saat (başlangıç) seçer.
3) Backend, seçilen slot için “o servisi yapabilen ve boş” personellerden birini otomatik atar.
4) Çakışma önlemi için transaction içinde re-check yapılarak randevu kaydedilir.

---

### Veri Modeli (SQLite)

Yeni tablolar
- staff
  - id, business_id (FK), name, active, created_at
- staff_services (çok-çok)
  - staff_id (FK), service_id (FK)

Mevcut tablo değişiklikleri
- appointments
  - start_time TEXT (HH:MM), end_time TEXT (HH:MM), staff_id INTEGER (FK)
- business_settings
  - business_id (PK),
  - slot_interval_minutes (default 15),
  - min_notice_minutes (default 60),
  - booking_window_days (default 30)

Notlar
- services.duration: servis süresi (dk).
- end_time = start_time + duration (buffer YOK).
- İşletme çalışma saatleri: businesses.opening_time, businesses.closing_time kullanılır.

---

### Owner Onboarding’da Alınacak Bilgiler

- İşletme temel: ad, tür, adres, telefon.
- Çalışma saatleri: opening_time, closing_time.
- Ayarlar (business_settings):
  - slot_interval_minutes: 15/20/30
  - min_notice_minutes: ör. 60
  - booking_window_days: ör. 30
- Servisler: name, duration (dk), price, description.
- Personel (staff):
  - name, active
  - Hangi servisleri yapabildiği (staff_services).

---

### Uygunluk (Availability) Hesaplama

Girdi
- business_id, service_id, date (YYYY-MM-DD)

Parametreler
- settings: slot_interval, min_notice
- business hours: opening_time, closing_time
- service.duration
- capable staff: staff_services’den o servisi yapabilen aktif personel listesi
- günün pending/confirmed randevuları

Adımlar
1) need_minutes = service.duration (buffer yok).
2) Zaman grid’i: opening_time → closing_time, slot_interval adımlarıyla dön.
3) Her slot için, “capable staff” listesinden o saat aralığında (slotStart → slotStart+need) çakışması olmayanları filtrele.
4) available_count = uygun staff sayısı. > 0 ise slot döndür.
5) min_notice: Bugün için, slot başlangıcı “şu an (GMT+3) + min_notice” sonrasında olmalı.
6) Response: UI yalnızca saat ve uygunluk sayısını görür (staff kimlikleri UI’a verilmez).

Örnek Response
```json
{
  "date": "2025-11-06",
  "slots": [
    { "time": "13:15", "available_count": 2 },
    { "time": "13:30", "available_count": 1 }
  ]
}
```

---

### Rezervasyon (Booking) Akışı

Girdi (POST /api/appointments)
- business_id, service_id, customer_id, date (YYYY-MM-DD), start_time (HH:MM)
- staff_id gönderilmez (otomatik atanacak).

Adımlar
1) need_minutes = duration; end_time hesapla.
2) capable staff listesini getir (staff_services).
3) Her staff için ilgili günde randevuları çek; slot ile çakışmayan ilk/uygun personeli seç.
   - Seçim stratejisi: en az randevulu veya ilk uygun.
4) Transaction (BEGIN IMMEDIATE):
   - Seçilen staff için tekrar çakışma kontrolü.
   - Uygunsa INSERT (appointments: staff_id, start_time, end_time, status='pending').
   - COMMIT; aksi halde ROLLBACK ve 409.
5) Response: atanan staff_id, start_time, end_time, status.

Örnek Request
```json
{
  "business_id": 12,
  "service_id": 5,
  "customer_id": 34,
  "date": "2025-11-06",
  "start_time": "13:15"
}
```

Örnek Response (201)
```json
{
  "staff_id": 8,
  "start_time": "13:15",
  "end_time": "14:00",
  "status": "pending"
}
```

Hata Örnekleri
- 409 Conflict: “No available staff for this slot” veya “Staff just became busy, pick another slot”
- 400 Bad Request: eksik parametre / servis bulunamadı

---

### UI/UX

- Personel seçimi yoktur (varsayılan: Herhangi bir personel).
- Slot listesi: “13:15 — 2 uygun personel” gibi rozet.
- Onay ekranında: “Personel, randevu oluşturulurken otomatik atanır.”
- İptal/yeniden planlama: atanan personel üzerinden aynı çakışma kuralları.

---

### Validasyon Kuralları

- start_time grid’e uymalı (slot_interval).
- start_time + duration, işletme kapanış saatini aşmamalı.
- min_notice sağlanmalı (GMT+3 referanslı “şu an”).
- service_id, business_id’ye ait olmalı.
- staff otomatik atanmadan önce “servisi yapabiliyor mu” ve “boş mu” kontrolü.
- status: pending, confirmed, cancelled, completed.

---

### Eşzamanlılık ve Tutarlılık

- Transaction akışı (SQLite):
  - BEGIN IMMEDIATE → re-check overlap → INSERT → COMMIT
- Overlap kontrolü (yarı-açık aralık [start, end)):
  - Çakışma: aStart < bEnd && bStart < aEnd
- Aynı anda iki istek geldiğinde ikinci istek 409 dönebilir.

---

### Endpoints (özet)

- GET /api/businesses/:id/availability?service_id=...&date=YYYY-MM-DD
  - Response: { date, slots: [{ time, available_count }] }
- POST /api/appointments
  - Body: { business_id, service_id, customer_id, date, start_time }
  - Behavior: otomatik staff seçimi, 201 veya 409/400.
- Personel Yönetimi (Owner Panel)
  - POST /api/businesses/:id/staff  { name }
  - GET  /api/businesses/:id/staff
  - POST /api/staff/:id/services    { service_ids: number[] }
- Ayarlar
  - POST/PUT /api/businesses/:id/settings { slot_interval_minutes, min_notice_minutes, booking_window_days }

---

### Migration Planı (adım adım)

1) business_settings tablosunu oluştur:
   - slot_interval_minutes, min_notice_minutes, booking_window_days
2) appointments tablosuna start_time, end_time, staff_id kolonlarını ekle.
   - Geçmiş veriler için: start_time = appointment_time, end_time = start_time + varsayılan duration (yoksa 60 dk varsayılabilir).
3) staff ve staff_services tablolarını oluştur; başlangıç personellerini ve servis yeteneklerini tanımla.
4) Availability endpoint’ini ekle; UI’da slot listelemesini buna bağla.
5) Appointments POST’ta otomatik atama ve transaction’lı re-check ekle.
6) UI’dan personel seçimi kaldır; bilgi metinlerini güncelle.

---

### Kenar Durumları

- Servis süresi uzun ve kapanışa yakın: slot üretiminde elenir.
- Eşzamanlı rezervasyon: transaction re-check ile 409.
- Personel pasif (active=0): otomatik atama havuzuna dahil edilmez.
- İptal: ilgili pencere boşalır; availability yeniden hesaplanınca yansır.
- Saatler sabit GMT+3 varsayımıyla hesaplanır; ayrı timezone alanı yoktur.

---

### Minimal Seçim Stratejisi (öneri)

- İlk uygun staff (basit ve hızlı), veya
- En az aktif randevu sayısına sahip staff (daha dengeli dağıtım).

Her iki yaklaşım da API sözleşmesini değiştirmez; yalnızca backend seçim fonksiyonunu değiştirir.

# TODO 3) Owner ana sayfasının kaldırılması (Owner sadece işletme paneli) — GMT+3 sabit

Amaç
- Owner (business_owner) hesapları “müşteri gibi” gezmesin ve dışarıdan randevu almasın.
- Owner giriş yaptığında doğrudan işletme yönetim paneline yönlendirilsin.

Kapsam
- Frontend
  - Route/Redirect: role === 'business_owner' için "/" → "/my-business" redirect.
  - Navbar: Owner rolünde “Ana Sayfa/Keşfet/Rezervasyon” gibi müşteri menüleri gizlenir.
  - BusinessDetail vs: Owner rolü bu sayfayı açsa bile müşteri rezervasyon formu gizlenir (yalnızca read-only bilgiler).
- Backend (opsiyonel sıkılaştırma)
  - POST /api/appointments (müşteri dalı): user.role === 'business_owner' ise 403 döndür (owner’ın müşteri akışını engelle).
  - Owner manuel booking (owner dalı) zaten yalnızca kendi işletmesine izin verir (ownership check ile).

Kabul Kriterleri
- [ ] Owner login sonrası "/" ziyaretinde otomatik "/my-business" sayfası açılır.
- [ ] Owner için navbar’da müşteri akışına ait menüler görünmez.
- [ ] Owner, müşteri akışından randevu oluşturamaz (FE gizli, BE 403 ile korumalı).
- [ ] Mevcut müşteri akışı etkilenmez.

Notlar
- Uygunluk (availability) endpoint’i müşteriye açık kalır, owner rolünde read-only erişim sorun değildir.
- Saat hesapları sabit GMT+3.




# TODO 4) Owner manuel randevuların otomatik onaylanması (status='confirmed') — GMT+3 sabit

Amaç
- Owner panelinden eklenen manuel randevular ek an onaya gerek duymadan “onaylandı” olarak kaydedilsin.

Kapsam
- Backend
  - POST /api/appointments (owner dalı):
    - Kayıt sırasında status varsayılanı 'confirmed' olacak.
    - source='owner_manual' korunur.
    - Diğer kurallar (slot grid, min_notice, çalışma saatleri, otomatik staff atama, transaction re-check) aynen devam eder.
- Frontend
  - Owner booking modalı başarı mesajında “Randevu onaylandı” ifadesi gösterilebilir.
  - Listede status rozeti “Onaylandı” olarak görünür (mevcut badge haritası ile uyumlu).

Kabul Kriterleri
- [ ] Owner manuel randevu oluşturduğunda API 201 döner ve body.status === 'confirmed' olur.
- [ ] Randevu listesinde “Onaylandı” rozeti ve “Owner” etiketi görünür.
- [ ] Otomatik personel atama ve çakışma önleme mantıkları değişmeden çalışır.

Test Senaryoları
- [ ] Owner manuel randevu: 201 + status='confirmed'.
- [ ] Eşzamanlı aynı slot iki istek: biri 201 (confirmed), diğeri 409.
- [ ] Owner müşteri dalından randevu denemesi: 403 (opsiyonel BE kuralı aktifse).

Notlar
- Müşteri randevuları 'pending' kalmaya devam eder (iş mantığına göre değiştirilebilir).
- Saat hesapları sabit GMT+3.

GitHub Copilot

# TODO 5) Localhost açılınca direkt authentication ekranından açılsın

Amaç
- Uygulama ilk açıldığında (localhost), kullanıcı oturumu yoksa doğrudan Login (/login) sayfasına yönlendirilsin.
- İsteğe bağlı: Geliştirme ortamında her açılışta “taze login” zorunlu olsun (persisted session olsa bile).

Kapsam
- Frontend
  - App.js: Global bir AuthGate ile authenticated=false durumunda login/register dışındaki tüm rotalarda otomatik /login yönlendirmesi.
  - Opsiyonel (sadece dev): REACT_APP_FORCE_FRESH_LOGIN=true ise uygulama ilk yüklemede mevcut oturumu düşürsün ve /login’a yönlendirsin (her açılışta yeniden giriş).
  - Login sonrası role bazlı yönlendirme korunur:
    - business_owner → /my-business
    - customer → /
  - PrivateRoute/BusinessOwnerRoute korumaları mevcut şekilde devam eder.

Kabul Kriterleri
- [ ] Misafir kullanıcı, localhost’u açtığında otomatik olarak /login’a yönlendirilir.
- [ ] REACT_APP_FORCE_FRESH_LOGIN=true iken, uygulama her açılışta (ilk yüklemede) login ekranına getirir (persisted session olsa dahi).
- [ ] REACT_APP_FORCE_FRESH_LOGIN=false iken, oturumu devam eden kullanıcılar kaldıkları sayfayı görebilir; misafirler /login’a gider.
- [ ] Login sonrası business_owner → /my-business, diğer roller → /.
- [ ] Route korumaları (PrivateRoute/BusinessOwnerRoute) beklenen şekilde çalışır.

Uygulama
- App.js içinde AuthGate mantığı:
````javascript
// ...existing code...
const AuthGate = () => {
  const { isAuthenticated, loading, logout } = useAuth();
  const location = useLocation();
  const navigate = useNavigate();
  const FORCE_FRESH = process.env.REACT_APP_FORCE_FRESH_LOGIN === 'true';
  const didRunRef = React.useRef(false);

  React.useEffect(() => {
    if (loading) return;

    // Dev seçeneği: her açılışta taze login
    if (!didRunRef.current && FORCE_FRESH) {
      didRunRef.current = true;
      if (isAuthenticated) logout?.();
      if (location.pathname !== '/login' && location.pathname !== '/register') {
        navigate('/login', { replace: true });
      }
      return;
    }

    // Standart: oturum yoksa login’a yönlendir
    if (!isAuthenticated && location.pathname !== '/login' && location.pathname !== '/register') {
      navigate('/login', { replace: true });
    }
  }, [loading, isAuthenticated, location.pathname, navigate, logout, FORCE_FRESH]);

  return null;
};
// ...existing code...
````

Notlar
- REACT_APP_FORCE_FRESH_LOGIN yalnızca geliştirme ortamında önerilir; prod’da kullanıcı deneyimini bozabilir.
- “Son kalınan sayfaya” dönüş davranışı, sadece kullanıcı zaten authenticated ise geçerlidir; misafir kullanıcılar daima /login’a yönlendirilir.

Test Senaryoları
- [ ] Misafir olarak localhost’u açın → /login.
- [ ] REACT_APP_FORCE_FRESH_LOGIN=true ile, önceden giriş yapmış olsanız da sayfayı yenileyince /login.
- [ ] REACT_APP_FORCE_FRESH_LOGIN=false ile, oturum varken kaldığınız sayfa; çıkış yaptıktan sonra yenileyince /login.
- [ ] Owner login → otomatik /my-business; müşteri login → /.